# Dechow's F-Score
## [Original Paper](https://poseidon01.ssrn.com/delivery.php?ID=556009073006105088000122086073105078024081022041024036031090115103120083014070085024099002021039108007045065097003086091003112104036001013036115108029089018008107000061062003088073093002097123117025122099081111118125007118076120069029016119120082027&EXT=pdf)

In [135]:
import calcbench as cb
import pandas as pd
import numpy as np

In [88]:
metrics = ['Assets', 
           'Liabilities',
           'CurrentAssets', 
           'Cash', 
           'ShortTermInvestments', 
           'CurrentLiabilities', 
           'CurrentLongTermDebt', 
           'LongTermDebt',
           'PreferredStockValue',
           'AccountsReceivable',
           'Inventory',
           'PPE',
           'NetIncome',
           'SharesIssued',
           'LoansReceivable',
           'LongTermInvestments',
           'Revenue']

In [52]:
tickers = cb.tickers(index='DJIA')

In [90]:
data = cb.standardized_data(company_identifiers=tickers, metrics=metrics, all_history=True, period_type='annual').fillna(0)

In [84]:
average_total_assets = ((data['Assets'] + data.shift(1)['Assets']) / 2)

In [123]:
#Working Capital
WC = data['CurrentAssets'] - data['CurrentLiabilities']

In [66]:
#Net Charge Offs
investments_and_advances = data['LoansReceivable'] # This is probably not totally correct.  The paper says Investments and Advances is DATA 32 from CompuStat
NCO = (data['Assets'] - data['CurrentAssets'] - investments_and_advances) - (data['Liabilities'] - data['CurrentLiabilities'] - data['LongTermDebt'])

In [67]:
FIN = (data['ShortTermInvestments'] + data['LongTermInvestments']) - (data['LongTermDebt'] + data['CurrentLongTermDebt'] + data['PreferredStockValue'])

In [126]:
RSST_Accruals = (WC.diff() + NCO.diff() + FIN.diff()) / average_total_assets

In [40]:
percent_soft_assets = (data['Assets'] - data['PPE']) / data['Assets']

In [60]:
issued_shares = data['SharesIssued'].diff() > 0

In [85]:
change_inventory = data['Inventory'].diff() / average_total_assets

In [86]:
change_receivables = data['AccountsReceivable'] / average_total_assets

In [115]:
cash_sales = data['Revenue'] - data['AccountsReceivable'].diff()

In [97]:
ROA = data['NetIncome'] / average_total_assets

In [87]:
intercept = -7.893

In [132]:
predicted_value = intercept + (.79 * RSST_Accruals) + (2.518 * change_receivables) + (1.191 * change_inventory) + (1.979 * percent_soft_assets) + (.171 * cash_sales.pct_change()) + (-.932 * ROA.diff()) + (1.029 * issued_shares)

In [139]:
probability = (np.e ** predicted_value) / (1 + np.e ** predicted_value)

In [140]:
probability

ticker,AAPL,AXP,BA,CAT,CSCO,CVX,DIS,DWDP,GS,HD,...,PFE,PG,TRV,UNH,UTX,V,VZ,WBA,WMT,XOM
period,,,,,,,,,,,,,,,,,,,,,
2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,0.0168354,0.00493558,0.00321948,0.00228419,NaN,0.00136196,0.00222985,0.00254057,NaN,0.00221872,...,0.0106664,0.00608339,NaN,0.00337975,0.00365471,0.00263488,0.00143836,NaN,0.000998201,0.00141211
2010,0.0149824,0.008808,0.00297072,0.00287523,0.00366721,0.00126998,0.00580404,0.00561341,NaN,0.00258362,...,0.00789151,0.00662782,NaN,0.00291258,0.00330719,0.00272078,0.00142516,0.00163872,0.000932677,0.00111933
2011,0.0159906,0.00339524,0.00287425,0.00327786,0.00311717,0.00125217,0.00196133,0.00551076,NaN,0.00087037,...,0.00751796,0.00609626,NaN,0.00295951,0.00932719,0.00279025,0.00144204,0.00153968,0.00102976,0.00107099
2012,NaN,0.00548956,0.00301572,0.00279833,0.00323581,0.00114871,0.00543624,0.00579916,NaN,0.00253508,...,0.00723701,0.00627842,NaN,0.00288808,0.00970322,0.0029894,0.00142858,0.00558872,0.00100385,0.000937984
2013,0.0105033,0.00552305,0.00309561,0.00230226,0.00969854,0.00102662,0.00215032,0.00652666,NaN,0.00243455,...,0.00720783,0.00226934,NaN,0.00279129,0.00918015,0.00246379,0.00160085,0.00169843,0.000983205,0.000896188
2014,0.00311858,0.00547169,0.00246328,0.00155998,0.00334774,0.000869087,0.00196768,0.00525821,NaN,0.00248577,...,0.00782496,0.00214428,NaN,0.00280087,0.00789437,0.00262225,0.00325332,0.00184225,0.000978173,0.000786849
2015,0.00369589,0.00565663,0.00262731,0.00169944,0.00342167,0.000760468,0.00192089,0.00476711,NaN,0.00285801,...,0.00665547,0.00202832,NaN,0.00295873,0.00801121,0.00277091,0.00157168,0.00788071,0.000970947,0.00068932
